In [0]:
## Hive
!pip install PyHive
!pip install thrift

In [2]:
!pip install pyspark
!pip install findspark
!pip install handyspark

     |████████████████████████████████| 215.7MB 63kB/s 
     |████████████████████████████████| 204kB 57.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=79bdbe6b3ca1ec67154ad1bc8d566bfd5eba51d63c61cb62307469650c8701f3
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
import requests
import datetime as dt
from time import sleep
from datetime import datetime, timedelta
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from pyspark.sql.types import *
from handyspark import *
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import Image
import numpy as np

In [0]:
JAVA_HOME="/usr/lib/jvm/java-1.8.0-openjdk-amd64"
SPARK_HOME = "/content/spark-2.3.2-bin-hadoop2.7"

In [5]:
!sudo update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [0]:
# Get URL
url ="http://waze-api.18.140.161.219.xip.io/"

In [0]:
#deff awal
class Get(object):
  def get_alldata(url, minutes_interval, day,hour_time,minute_time):
    def get_time(minutes_interval,day,hour_time,minute_time):
      current = dt.datetime(2019,7,day,hour_time,minute_time,0)
      endcurr = current + dt.timedelta(minutes=minutes_interval)
      return (current.strftime("%Y-%m-%dT%H:%M:%S"),endcurr.strftime("%Y-%m-%dT%H:%M:%S"))
    start_time, end_time = get_time(minutes_interval,day,hour_time,minute_time)
    fullurl = url+'?start_time='+start_time+'&end_time='+end_time
    response = requests.get(fullurl)
    return response.json()
  
  '''convert data into dataframe
    only take alerts,irregularities,jams'''

  def get_address(lat,long):
    geolocator = Nominatim(user_agent="waze")
    latlong = str(lat)+","+str(long)
    location = geolocator.reverse(latlong)
    return location.address

  def drop_listval(df):
    lst = []
    for col in df:
        if isinstance(df[col][0],list)==True:
            lst.append(col)
    return df.drop(columns = list(set(lst)))

In [0]:
# Crawl data
data_crawl = Get.get_alldata(url, 10080 ,23, 13, 30)  

In [9]:
#bikin list waktu
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

date_time = [dt.strftime('%Y-%m-%dT%H:%M:%S') for dt in 
        datetime_range(datetime(2019, 7, 23, 0, 0, 0), 
                       datetime(2019, 7, 30, 1, 0, 0), 
                       timedelta(hours=3))]

date_time

['2019-07-23T00:00:00',
 '2019-07-23T03:00:00',
 '2019-07-23T06:00:00',
 '2019-07-23T09:00:00',
 '2019-07-23T12:00:00',
 '2019-07-23T15:00:00',
 '2019-07-23T18:00:00',
 '2019-07-23T21:00:00',
 '2019-07-24T00:00:00',
 '2019-07-24T03:00:00',
 '2019-07-24T06:00:00',
 '2019-07-24T09:00:00',
 '2019-07-24T12:00:00',
 '2019-07-24T15:00:00',
 '2019-07-24T18:00:00',
 '2019-07-24T21:00:00',
 '2019-07-25T00:00:00',
 '2019-07-25T03:00:00',
 '2019-07-25T06:00:00',
 '2019-07-25T09:00:00',
 '2019-07-25T12:00:00',
 '2019-07-25T15:00:00',
 '2019-07-25T18:00:00',
 '2019-07-25T21:00:00',
 '2019-07-26T00:00:00',
 '2019-07-26T03:00:00',
 '2019-07-26T06:00:00',
 '2019-07-26T09:00:00',
 '2019-07-26T12:00:00',
 '2019-07-26T15:00:00',
 '2019-07-26T18:00:00',
 '2019-07-26T21:00:00',
 '2019-07-27T00:00:00',
 '2019-07-27T03:00:00',
 '2019-07-27T06:00:00',
 '2019-07-27T09:00:00',
 '2019-07-27T12:00:00',
 '2019-07-27T15:00:00',
 '2019-07-27T18:00:00',
 '2019-07-27T21:00:00',
 '2019-07-28T00:00:00',
 '2019-07-28T03:

In [10]:
# Joining per Batch
week_list = []
df_alerts, df_jams, df_irregularities = pd.DataFrame({}), pd.DataFrame({}), pd.DataFrame({})
for i in date_time:
  try :
    daydate = int(i[8:10])
    dayhour = int(i[11:13])
    dayminute = int(i[14:16])
    week_list.append(data_crawl)
    print(daydate)
    for i in tqdm_notebook(range (56)): # 7 Days Loop
      df_alerts = pd.concat([df_alerts,json_normalize(week_list[i]['alerts'])], sort=True, ignore_index=True)
      df_jams = pd.concat([df_jams,json_normalize(week_list[i]['jams'])], sort=True, ignore_index=True)
      df_irregularities = pd.concat([df_irregularities,json_normalize(week_list[i]['irregularities'])], sort=True, ignore_index=True)
  except :
    pass

23



23



23



23



23


...

29


29


29


29


30


In [18]:
df_alerts

,city,confidence,country,created_time,end_time,kode_kabupaten,location,magvar,nThumbsUp,nama_kabupaten,pubMillis,reliability,reportRating,roadType,start_time,street,subtype,type,uuid
32,Depok,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.90008, -6.414294]",243,0,KAB. BOGOR,1564361553000,5,0,2,2019-07-29T08:13:00,Leuwinanggung,JAM_STAND_STILL_TRAFFIC,JAM,5b088b7e-ada4-3233-9f3b-840e378b8c33
42,Bekasi,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.924031, -6.381164]",313,0,KAB. BOGOR,1564361239000,5,3,7,2019-07-29T08:13:00,Trans Yogi,JAM_HEAVY_TRAFFIC,JAM,895ab6d1-7988-36c6-9a27-d063c85ec5fa
46,Bogor,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.829785, -6.636852]",123,0,KAB. BOGOR,1564361849000,5,2,6,2019-07-29T08:13:00,N2 Jalan Raya Tajur,JAM_STAND_STILL_TRAFFIC,JAM,85052fa5-adab-3de3-b3e8-5b67cbac2784
47,Bogor,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.830287, -6.637181]",123,0,KAB. BOGOR,1564361969000,5,2,6,2019-07-29T08:13:00,N2 Jalan Raya Tajur,JAM_STAND_STILL_TRAFFIC,JAM,9d8e1b86-9d50-38a0-ab81-a61fff812ee9
48,Bogor,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.832283, -6.638482]",122,0,KAB. BOGOR,1564362403000,5,2,6,2019-07-29T08:13:00,N2 Jalan Raya Tajur,JAM_STAND_STILL_TRAFFIC,JAM,98ab9e4f-ce51-3a3d-838d-e59bc7995b6b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321482,Depok,0,ID,2019-10-04T22:28:45,2019-07-26T08:25:00,32.76,"[106.780116, -6.316543]",81,0,KOTA DEPOK,1564102627000,5,3,1,2019-07-26T08:24:00,Vila Cinere Mas,JAM_HEAVY_TRAFFIC,JAM,8699a73e-c03c-3400-a6b1-facd3f819310
1321485,Depok,0,ID,2019-10-04T22:28:45,2019-07-26T08:25:00,32.76,"[106.780515, -6.338912]",28,0,KOTA DEPOK,1564104072000,5,1,7,2019-07-26T08:24:00,Cinere Raya,JAM_STAND_STILL_TRAFFIC,JAM,3a95576c-0c70-358c-9f69-16e638bbc86e
1321518,Bekasi,0,ID,2019-10-04T22:30:29,2019-07-26T08:26:00,32.01,"[106.955172, -6.308929]",3,0,KAB. BOGOR,1564103762000,5,2,2,2019-07-26T08:25:00,Wibawa Mukti 2,JAM_HEAVY_TRAFFIC,JAM,04956d73-335f-32ea-9176-a6d77df9fdf7
1321522,Bekasi,0,ID,2019-10-04T22:30:29,2019-07-26T08:26:00,32.01,"[106.983453, -6.304393]",177,0,KAB. BOGOR,1564103694000,5,3,7,2019-07-26T08:25:00,Narogong Raya,JAM_HEAVY_TRAFFIC,JAM,b5fd814a-63d7-3471-b4b5-201edf2d211c


In [19]:
df_jams

,city,country,created_time,delay,endNode,end_time,kode_kabupaten,length,level,line,nama_kabupaten,pubMillis,roadType,speed,speedKMH,startNode,start_time,street,turnType,type,uuid
0,Bogor,ID,2019-10-04T15:13:41,267,Cipaku Raya,2019-07-24T07:45:00,32.01,251,4,"[[106.80851, -6.628578], [106.808563, -6.62839...",KAB. BOGOR,1563927699475,2,0.802778,2.89,,2019-07-24T07:44:00,Ence Sumartadireja,NONE,NONE,2004991111
2,Bogor,ID,2019-10-04T15:13:41,142,Jalan Danau Bogor Raya,2019-07-24T07:45:00,32.01,214,4,"[[106.815947, -6.598947], [106.816403, -6.5986...",KAB. BOGOR,1563928807539,2,1.261110,4.54,,2019-07-24T07:44:00,Padi,NONE,NONE,1917138736
14,Bogor,ID,2019-10-04T15:13:41,263,N11 Cifor,2019-07-24T07:45:00,32.01,1921,2,"[[106.739543, -6.570723], [106.739895, -6.5715...",KAB. BOGOR,1563928518156,6,4.150000,14.94,,2019-07-24T07:44:00,N11 Jalan Raya Dramaga,NONE,NONE,1918209258
16,Bogor,ID,2019-10-04T15:13:41,360,N11 Jalan Raya Dramaga,2019-07-24T07:45:00,32.01,1238,3,"[[106.749781, -6.572456], [106.749633, -6.5723...",KAB. BOGOR,1563928586831,6,2.527780,9.10,,2019-07-24T07:44:00,N11 Jalan Raya Dramaga,NONE,Small,1959692650
18,Bogor,ID,2019-10-04T15:13:41,179,N11 KH Soleh Iskandar,2019-07-24T07:45:00,32.01,1190,3,"[[106.773078, -6.543969], [106.773611, -6.5451...",KAB. BOGOR,1563928659214,6,4.158330,14.97,,2019-07-24T07:44:00,N12 Jalan Raya Baru,NONE,NONE,1943399863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321582,Bekasi,ID,2019-10-04T15:35:01,252,Swatantra 5,2019-07-24T07:46:00,32.75,380,4,"[[106.973611, -6.299099], [106.972597, -6.2989...",KOTA BEKASI,1563922541806,2,1.266670,4.56,,2019-07-24T07:45:00,Cipendawa Baru,NONE,NONE,1885419089
1321584,Bekasi,ID,2019-10-04T15:35:01,574,,2019-07-24T07:46:00,32.75,531,4,"[[106.983388, -6.302821], [106.983416, -6.3023...",KOTA BEKASI,1563927184043,7,0.838889,3.02,,2019-07-24T07:45:00,Narogong Raya,NONE,Small,1957988010
1321589,Bekasi,ID,2019-10-04T15:35:01,153,Mustikasari Raya,2019-07-24T07:46:00,32.75,222,3,"[[107.018498, -6.27669], [107.017881, -6.27663...",KOTA BEKASI,1563929248839,2,1.166670,4.20,,2019-07-24T07:45:00,Pondok Timur Indah,NONE,NONE,1919255158
1321596,Bekasi,ID,2019-10-04T15:35:01,174,Jalan Jakasetia,2019-07-24T07:46:00,32.75,964,3,"[[106.986776, -6.263594], [106.986455, -6.2644...",KOTA BEKASI,1563924088734,2,3.380560,12.17,,2019-07-24T07:45:00,Pekayon Jaya,NONE,NONE,1910619726


In [20]:
df_irregularities

,alertCount,alerts,city,country,created_time,delaySeconds,detectionDate,detectionDateMillis,driversCount,endNode,end_time,highway,id,jamLevel,kode_kabupaten,length,line,nComments,nImages,nThumbsUp,nama_kabupaten,regularSpeed,seconds,severity,speed,startNode,start_time,street,trend,type,updateDate,updateDateMillis
6,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,237,Wed Jul 24 00:56:00 +0000 2019,1563929760560,23,Null,2019-07-24T08:21:00,False,326929,3,32.75,1288,"[[106.954166, -6.308456], [106.954215, -6.3084...",0,0,0,KOTA BEKASI,15.01,389,5,11.89,Null,2019-07-24T08:20:00,Wibawa Mukti 2,1,Small,2019-07-24T01:23:00,1563931380527
7,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,688,Wed Jul 24 00:59:42 +0000 2019,1563929982325,15,Null,2019-07-24T08:21:00,False,326937,4,32.75,1341,"[[106.948313, -6.321199], [106.948655, -6.3202...",0,0,0,KOTA BEKASI,25.59,820,5,5.88,Null,2019-07-24T08:20:00,Wibawa Mukti 2,0,Medium,2019-07-24T01:23:00,1563931380430
8,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,357,Wed Jul 24 01:00:49 +0000 2019,1563930049541,14,Null,2019-07-24T08:21:00,False,326943,3,32.75,1809,"[[106.951186, -6.279266], [106.951249, -6.2791...",0,0,0,KOTA BEKASI,21.32,552,5,11.79,Null,2019-07-24T08:20:00,Ratna,-1,Small,2019-07-24T01:23:00,1563931380510
9,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,949,Wed Jul 24 01:22:53 +0000 2019,1563931373558,10,Null,2019-07-24T08:21:00,False,326954,4,32.75,986,"[[106.980454, -6.269301], [106.980006, -6.2691...",0,0,0,KOTA BEKASI,16.00,1149,5,3.08,Null,2019-07-24T08:20:00,Pulo Ribung,1,Large,2019-07-24T01:23:00,1563931380567
10,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,141,Wed Jul 24 00:38:59 +0000 2019,1563928739736,13,Null,2019-07-24T08:21:00,False,326960,3,32.75,722,"[[106.978561, -6.300514], [106.978645, -6.3005...",0,0,0,KOTA BEKASI,32.99,218,5,11.90,Null,2019-07-24T08:20:00,Cipendawa Baru,-1,Small,2019-07-24T01:23:00,1563931380419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321582,0,[],Depok,ID,2019-10-04T23:49:49,675,Mon Jul 29 23:48:46 +0000 2019,1564444126300,29,Null,2019-07-30T08:21:00,False,336304,4,32.76,1021,"[[106.791516, -6.352186], [106.791563, -6.3520...",0,0,0,KOTA DEPOK,8.48,820,5,4.47,Null,2019-07-30T08:20:00,Krukut Raya,0,Small,2019-07-30T01:22:44,1564449764942
1321588,0,[],Bekasi,ID,2019-10-05T02:35:24,222,Tue Jul 30 00:28:39 +0000 2019,1564446519770,20,Null,2019-07-30T08:33:00,False,359479,3,32.16,1180,"[[107.017011, -6.321264], [107.01703, -6.32126...",0,0,0,KAB. BEKASI,28.75,344,5,12.33,Null,2019-07-30T08:32:00,Letjen Suprapto,0,Small,2019-07-30T01:34:03,1564450443083
1321596,0,[],Bekasi,ID,2019-10-05T02:37:25,468,Tue Jul 30 01:26:28 +0000 2019,1564449988033,2,Null,2019-07-30T08:33:00,False,359722,4,32.75,696,"[[106.957727, -6.256052], [106.957754, -6.2558...",0,0,0,KOTA BEKASI,13.93,555,5,4.51,Null,2019-07-30T08:32:00,Arah Tol Jkt-Cikampek/Bekasi/Tol Purbaleunyi,-1,Small,2019-07-30T01:34:02,1564450442983
1321597,0,[],Bekasi,ID,2019-10-05T02:37:25,222,Tue Jul 30 00:28:39 +0000 2019,1564446519770,20,Null,2019-07-30T08:33:00,False,359729,3,32.75,1180,"[[107.017011, -6.321264], [107.01703, -6.32126...",0,0,0,KOTA BEKASI,28.75,344,5,12.33,Null,2019-07-30T08:32:00,Letjen Suprapto,0,Small,2019-07-30T01:34:03,1564450443083


In [21]:
#### Data Preprocessing
### Data Normalization & Filtering
## Alert
# Filter 5 City
df_alerts = df_alerts.loc[df_alerts['city'].isin(['Bekasi', 'Depok', 'Bogor', 'Purwakarta', 'Karawang'])]
# Filter Jam Subtype
df_alerts = df_alerts[(df_alerts['subtype'] == 'JAM_HEAVY_TRAFFIC') | (df_alerts['subtype'] == 'JAM_STAND_STILL_TRAFFIC')]

df_alerts

,city,confidence,country,created_time,end_time,kode_kabupaten,location,magvar,nThumbsUp,nama_kabupaten,pubMillis,reliability,reportRating,roadType,start_time,street,subtype,type,uuid
32,Depok,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.90008, -6.414294]",243,0,KAB. BOGOR,1564361553000,5,0,2,2019-07-29T08:13:00,Leuwinanggung,JAM_STAND_STILL_TRAFFIC,JAM,5b088b7e-ada4-3233-9f3b-840e378b8c33
42,Bekasi,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.924031, -6.381164]",313,0,KAB. BOGOR,1564361239000,5,3,7,2019-07-29T08:13:00,Trans Yogi,JAM_HEAVY_TRAFFIC,JAM,895ab6d1-7988-36c6-9a27-d063c85ec5fa
46,Bogor,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.829785, -6.636852]",123,0,KAB. BOGOR,1564361849000,5,2,6,2019-07-29T08:13:00,N2 Jalan Raya Tajur,JAM_STAND_STILL_TRAFFIC,JAM,85052fa5-adab-3de3-b3e8-5b67cbac2784
47,Bogor,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.830287, -6.637181]",123,0,KAB. BOGOR,1564361969000,5,2,6,2019-07-29T08:13:00,N2 Jalan Raya Tajur,JAM_STAND_STILL_TRAFFIC,JAM,9d8e1b86-9d50-38a0-ab81-a61fff812ee9
48,Bogor,0,ID,2019-10-04T22:26:59,2019-07-29T08:14:00,32.01,"[106.832283, -6.638482]",122,0,KAB. BOGOR,1564362403000,5,2,6,2019-07-29T08:13:00,N2 Jalan Raya Tajur,JAM_STAND_STILL_TRAFFIC,JAM,98ab9e4f-ce51-3a3d-838d-e59bc7995b6b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321482,Depok,0,ID,2019-10-04T22:28:45,2019-07-26T08:25:00,32.76,"[106.780116, -6.316543]",81,0,KOTA DEPOK,1564102627000,5,3,1,2019-07-26T08:24:00,Vila Cinere Mas,JAM_HEAVY_TRAFFIC,JAM,8699a73e-c03c-3400-a6b1-facd3f819310
1321485,Depok,0,ID,2019-10-04T22:28:45,2019-07-26T08:25:00,32.76,"[106.780515, -6.338912]",28,0,KOTA DEPOK,1564104072000,5,1,7,2019-07-26T08:24:00,Cinere Raya,JAM_STAND_STILL_TRAFFIC,JAM,3a95576c-0c70-358c-9f69-16e638bbc86e
1321518,Bekasi,0,ID,2019-10-04T22:30:29,2019-07-26T08:26:00,32.01,"[106.955172, -6.308929]",3,0,KAB. BOGOR,1564103762000,5,2,2,2019-07-26T08:25:00,Wibawa Mukti 2,JAM_HEAVY_TRAFFIC,JAM,04956d73-335f-32ea-9176-a6d77df9fdf7
1321522,Bekasi,0,ID,2019-10-04T22:30:29,2019-07-26T08:26:00,32.01,"[106.983453, -6.304393]",177,0,KAB. BOGOR,1564103694000,5,3,7,2019-07-26T08:25:00,Narogong Raya,JAM_HEAVY_TRAFFIC,JAM,b5fd814a-63d7-3471-b4b5-201edf2d211c


In [22]:
## Jams
# Filter 5 City
df_jams = df_jams.loc[df_jams['city'].isin(['Bekasi', 'Depok', 'Bogor', 'Purwakarta', 'Karawang'])]
# Filter length max 3000 meters
df_jams = df_jams[(df_jams['length'] <= 3000)]
# Fillna Delay Median
df_jams['delay'] = df_jams['delay'].apply(pd.to_numeric, errors='coerce')
df_jams['delay'] = df_jams['delay'].fillna(df_jams['delay'].median())
# Filter Delay by Median
df_jams = df_jams[(df_jams['delay'] > df_jams['delay'].median())]
df_jams

,city,country,created_time,delay,endNode,end_time,kode_kabupaten,length,level,line,nama_kabupaten,pubMillis,roadType,speed,speedKMH,startNode,start_time,street,turnType,type,uuid
0,Bogor,ID,2019-10-04T15:13:41,267,Cipaku Raya,2019-07-24T07:45:00,32.01,251,4,"[[106.80851, -6.628578], [106.808563, -6.62839...",KAB. BOGOR,1563927699475,2,0.802778,2.89,,2019-07-24T07:44:00,Ence Sumartadireja,NONE,NONE,2004991111
14,Bogor,ID,2019-10-04T15:13:41,263,N11 Cifor,2019-07-24T07:45:00,32.01,1921,2,"[[106.739543, -6.570723], [106.739895, -6.5715...",KAB. BOGOR,1563928518156,6,4.150000,14.94,,2019-07-24T07:44:00,N11 Jalan Raya Dramaga,NONE,NONE,1918209258
16,Bogor,ID,2019-10-04T15:13:41,360,N11 Jalan Raya Dramaga,2019-07-24T07:45:00,32.01,1238,3,"[[106.749781, -6.572456], [106.749633, -6.5723...",KAB. BOGOR,1563928586831,6,2.527780,9.10,,2019-07-24T07:44:00,N11 Jalan Raya Dramaga,NONE,Small,1959692650
58,Bekasi,ID,2019-10-04T15:13:41,935,Trans Yogi,2019-07-24T07:45:00,32.01,2061,4,"[[106.917963, -6.376423], [106.918445, -6.3766...",KAB. BOGOR,1563924802057,7,1.808330,6.51,,2019-07-24T07:44:00,Trans Yogi,NONE,Large,1915082677
66,Bekasi,ID,2019-10-04T15:13:41,346,Trans Yogi,2019-07-24T07:45:00,32.01,154,4,"[[106.929829, -6.3808], [106.929516, -6.382159]]",KAB. BOGOR,1563928808016,1,0.408333,1.47,,2019-07-24T07:44:00,Lurah Namat,NONE,NONE,1917139456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321570,Bekasi,ID,2019-10-04T15:35:01,239,Sinar Asih 2,2019-07-24T07:46:00,32.75,1345,3,"[[106.958967, -6.296426], [106.958413, -6.2979...",KOTA BEKASI,1563927182614,2,3.433330,12.36,,2019-07-24T07:45:00,Wibawa Mukti 2,NONE,NONE,1953027632
1321577,Bekasi,ID,2019-10-04T15:35:01,382,Asem Jaya,2019-07-24T07:46:00,32.75,600,3,"[[107.016856, -6.297115], [107.017344, -6.2972...",KOTA BEKASI,1563928369295,2,1.241670,4.47,,2019-07-24T07:45:00,Asem Raya,NONE,Small,1917970858
1321581,Bekasi,ID,2019-10-04T15:35:01,336,,2019-07-24T07:46:00,32.75,1001,3,"[[106.984741, -6.292109], [106.984727, -6.2922...",KOTA BEKASI,1563927256248,7,2.286110,8.23,,2019-07-24T07:45:00,Narogong Raya,NONE,NONE,2004415771
1321582,Bekasi,ID,2019-10-04T15:35:01,252,Swatantra 5,2019-07-24T07:46:00,32.75,380,4,"[[106.973611, -6.299099], [106.972597, -6.2989...",KOTA BEKASI,1563922541806,2,1.266670,4.56,,2019-07-24T07:45:00,Cipendawa Baru,NONE,NONE,1885419089


In [23]:
## Irregularities
# Filter 5 City
df_irregularities = df_irregularities.loc[df_irregularities['city'].isin(['Bekasi', 'Depok', 'Bogor', 'Purwakarta', 'Karawang'])]
# Filter length max 3000 meters
df_irregularities = df_irregularities[(df_irregularities['length'] <= 3000)]
# Filter highway == False
df_irregularities = df_irregularities[(df_irregularities['highway'] == False)]

df_irregularities

,alertCount,alerts,city,country,created_time,delaySeconds,detectionDate,detectionDateMillis,driversCount,endNode,end_time,highway,id,jamLevel,kode_kabupaten,length,line,nComments,nImages,nThumbsUp,nama_kabupaten,regularSpeed,seconds,severity,speed,startNode,start_time,street,trend,type,updateDate,updateDateMillis
6,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,237,Wed Jul 24 00:56:00 +0000 2019,1563929760560,23,Null,2019-07-24T08:21:00,False,326929,3,32.75,1288,"[[106.954166, -6.308456], [106.954215, -6.3084...",0,0,0,KOTA BEKASI,15.01,389,5,11.89,Null,2019-07-24T08:20:00,Wibawa Mukti 2,1,Small,2019-07-24T01:23:00,1563931380527
7,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,688,Wed Jul 24 00:59:42 +0000 2019,1563929982325,15,Null,2019-07-24T08:21:00,False,326937,4,32.75,1341,"[[106.948313, -6.321199], [106.948655, -6.3202...",0,0,0,KOTA BEKASI,25.59,820,5,5.88,Null,2019-07-24T08:20:00,Wibawa Mukti 2,0,Medium,2019-07-24T01:23:00,1563931380430
8,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,357,Wed Jul 24 01:00:49 +0000 2019,1563930049541,14,Null,2019-07-24T08:21:00,False,326943,3,32.75,1809,"[[106.951186, -6.279266], [106.951249, -6.2791...",0,0,0,KOTA BEKASI,21.32,552,5,11.79,Null,2019-07-24T08:20:00,Ratna,-1,Small,2019-07-24T01:23:00,1563931380510
9,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,949,Wed Jul 24 01:22:53 +0000 2019,1563931373558,10,Null,2019-07-24T08:21:00,False,326954,4,32.75,986,"[[106.980454, -6.269301], [106.980006, -6.2691...",0,0,0,KOTA BEKASI,16.00,1149,5,3.08,Null,2019-07-24T08:20:00,Pulo Ribung,1,Large,2019-07-24T01:23:00,1563931380567
10,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,141,Wed Jul 24 00:38:59 +0000 2019,1563928739736,13,Null,2019-07-24T08:21:00,False,326960,3,32.75,722,"[[106.978561, -6.300514], [106.978645, -6.3005...",0,0,0,KOTA BEKASI,32.99,218,5,11.90,Null,2019-07-24T08:20:00,Cipendawa Baru,-1,Small,2019-07-24T01:23:00,1563931380419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321582,0,[],Depok,ID,2019-10-04T23:49:49,675,Mon Jul 29 23:48:46 +0000 2019,1564444126300,29,Null,2019-07-30T08:21:00,False,336304,4,32.76,1021,"[[106.791516, -6.352186], [106.791563, -6.3520...",0,0,0,KOTA DEPOK,8.48,820,5,4.47,Null,2019-07-30T08:20:00,Krukut Raya,0,Small,2019-07-30T01:22:44,1564449764942
1321588,0,[],Bekasi,ID,2019-10-05T02:35:24,222,Tue Jul 30 00:28:39 +0000 2019,1564446519770,20,Null,2019-07-30T08:33:00,False,359479,3,32.16,1180,"[[107.017011, -6.321264], [107.01703, -6.32126...",0,0,0,KAB. BEKASI,28.75,344,5,12.33,Null,2019-07-30T08:32:00,Letjen Suprapto,0,Small,2019-07-30T01:34:03,1564450443083
1321596,0,[],Bekasi,ID,2019-10-05T02:37:25,468,Tue Jul 30 01:26:28 +0000 2019,1564449988033,2,Null,2019-07-30T08:33:00,False,359722,4,32.75,696,"[[106.957727, -6.256052], [106.957754, -6.2558...",0,0,0,KOTA BEKASI,13.93,555,5,4.51,Null,2019-07-30T08:32:00,Arah Tol Jkt-Cikampek/Bekasi/Tol Purbaleunyi,-1,Small,2019-07-30T01:34:02,1564450442983
1321597,0,[],Bekasi,ID,2019-10-05T02:37:25,222,Tue Jul 30 00:28:39 +0000 2019,1564446519770,20,Null,2019-07-30T08:33:00,False,359729,3,32.75,1180,"[[107.017011, -6.321264], [107.01703, -6.32126...",0,0,0,KOTA BEKASI,28.75,344,5,12.33,Null,2019-07-30T08:32:00,Letjen Suprapto,0,Small,2019-07-30T01:34:03,1564450443083


In [0]:
### Feature Engineering
#### If Delay >= and level >= 3, Peak is True 
##### Jams
df_jams['peak'] = np.where(np.logical_and(df_jams['delay'] >= df_jams['delay'].median(), df_jams['level'] >= 3), 1, 0)

# Peak Level Irregularities
df_irregularities['isPeak'] = np.where(np.logical_and(df_irregularities['delaySeconds'] >= df_irregularities['delaySeconds'].median(), df_irregularities['jamLevel'] >= 3), 1, 0)

# BK Driver by DriverCount
df_irregularities['BK_driverCount'] = df_irregularities['driversCount'] * 0.25
df_irregularities['BK_driverCount'] = df_irregularities['BK_driverCount'].astype('int64')

In [45]:
df_jams

,city,country,created_time,delay,endNode,end_time,kode_kabupaten,length,level,line,nama_kabupaten,pubMillis,roadType,speed,speedKMH,startNode,start_time,street,turnType,type,uuid,peak
0,Bogor,ID,2019-10-04T15:13:41,267,Cipaku Raya,2019-07-24T07:45:00,32.01,251,4,"[[106.80851, -6.628578], [106.808563, -6.62839...",KAB. BOGOR,1563927699475,2,0.802778,2.89,,2019-07-24T07:44:00,Ence Sumartadireja,NONE,NONE,2004991111,0
14,Bogor,ID,2019-10-04T15:13:41,263,N11 Cifor,2019-07-24T07:45:00,32.01,1921,2,"[[106.739543, -6.570723], [106.739895, -6.5715...",KAB. BOGOR,1563928518156,6,4.150000,14.94,,2019-07-24T07:44:00,N11 Jalan Raya Dramaga,NONE,NONE,1918209258,0
16,Bogor,ID,2019-10-04T15:13:41,360,N11 Jalan Raya Dramaga,2019-07-24T07:45:00,32.01,1238,3,"[[106.749781, -6.572456], [106.749633, -6.5723...",KAB. BOGOR,1563928586831,6,2.527780,9.10,,2019-07-24T07:44:00,N11 Jalan Raya Dramaga,NONE,Small,1959692650,1
58,Bekasi,ID,2019-10-04T15:13:41,935,Trans Yogi,2019-07-24T07:45:00,32.01,2061,4,"[[106.917963, -6.376423], [106.918445, -6.3766...",KAB. BOGOR,1563924802057,7,1.808330,6.51,,2019-07-24T07:44:00,Trans Yogi,NONE,Large,1915082677,1
66,Bekasi,ID,2019-10-04T15:13:41,346,Trans Yogi,2019-07-24T07:45:00,32.01,154,4,"[[106.929829, -6.3808], [106.929516, -6.382159]]",KAB. BOGOR,1563928808016,1,0.408333,1.47,,2019-07-24T07:44:00,Lurah Namat,NONE,NONE,1917139456,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321570,Bekasi,ID,2019-10-04T15:35:01,239,Sinar Asih 2,2019-07-24T07:46:00,32.75,1345,3,"[[106.958967, -6.296426], [106.958413, -6.2979...",KOTA BEKASI,1563927182614,2,3.433330,12.36,,2019-07-24T07:45:00,Wibawa Mukti 2,NONE,NONE,1953027632,0
1321577,Bekasi,ID,2019-10-04T15:35:01,382,Asem Jaya,2019-07-24T07:46:00,32.75,600,3,"[[107.016856, -6.297115], [107.017344, -6.2972...",KOTA BEKASI,1563928369295,2,1.241670,4.47,,2019-07-24T07:45:00,Asem Raya,NONE,Small,1917970858,1
1321581,Bekasi,ID,2019-10-04T15:35:01,336,,2019-07-24T07:46:00,32.75,1001,3,"[[106.984741, -6.292109], [106.984727, -6.2922...",KOTA BEKASI,1563927256248,7,2.286110,8.23,,2019-07-24T07:45:00,Narogong Raya,NONE,NONE,2004415771,1
1321582,Bekasi,ID,2019-10-04T15:35:01,252,Swatantra 5,2019-07-24T07:46:00,32.75,380,4,"[[106.973611, -6.299099], [106.972597, -6.2989...",KOTA BEKASI,1563922541806,2,1.266670,4.56,,2019-07-24T07:45:00,Cipendawa Baru,NONE,NONE,1885419089,0


In [46]:
df_irregularities

,alertCount,alerts,city,country,created_time,delaySeconds,detectionDate,detectionDateMillis,driversCount,endNode,end_time,highway,id,jamLevel,kode_kabupaten,length,line,nComments,nImages,nThumbsUp,nama_kabupaten,regularSpeed,seconds,severity,speed,startNode,start_time,street,trend,type,updateDate,updateDateMillis,isPeak,BK_driverCount
6,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,237,Wed Jul 24 00:56:00 +0000 2019,1563929760560,23,Null,2019-07-24T08:21:00,False,326929,3,32.75,1288,"[[106.954166, -6.308456], [106.954215, -6.3084...",0,0,0,KOTA BEKASI,15.01,389,5,11.89,Null,2019-07-24T08:20:00,Wibawa Mukti 2,1,Small,2019-07-24T01:23:00,1563931380527,0,5
7,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,688,Wed Jul 24 00:59:42 +0000 2019,1563929982325,15,Null,2019-07-24T08:21:00,False,326937,4,32.75,1341,"[[106.948313, -6.321199], [106.948655, -6.3202...",0,0,0,KOTA BEKASI,25.59,820,5,5.88,Null,2019-07-24T08:20:00,Wibawa Mukti 2,0,Medium,2019-07-24T01:23:00,1563931380430,1,3
8,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,357,Wed Jul 24 01:00:49 +0000 2019,1563930049541,14,Null,2019-07-24T08:21:00,False,326943,3,32.75,1809,"[[106.951186, -6.279266], [106.951249, -6.2791...",0,0,0,KOTA BEKASI,21.32,552,5,11.79,Null,2019-07-24T08:20:00,Ratna,-1,Small,2019-07-24T01:23:00,1563931380510,0,3
9,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,949,Wed Jul 24 01:22:53 +0000 2019,1563931373558,10,Null,2019-07-24T08:21:00,False,326954,4,32.75,986,"[[106.980454, -6.269301], [106.980006, -6.2691...",0,0,0,KOTA BEKASI,16.00,1149,5,3.08,Null,2019-07-24T08:20:00,Pulo Ribung,1,Large,2019-07-24T01:23:00,1563931380567,1,2
10,23,"[{'city': 'Bekasi', 'confidence': 0, 'country'...",Bekasi,ID,2019-10-04T22:33:50,141,Wed Jul 24 00:38:59 +0000 2019,1563928739736,13,Null,2019-07-24T08:21:00,False,326960,3,32.75,722,"[[106.978561, -6.300514], [106.978645, -6.3005...",0,0,0,KOTA BEKASI,32.99,218,5,11.90,Null,2019-07-24T08:20:00,Cipendawa Baru,-1,Small,2019-07-24T01:23:00,1563931380419,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1321582,0,[],Depok,ID,2019-10-04T23:49:49,675,Mon Jul 29 23:48:46 +0000 2019,1564444126300,29,Null,2019-07-30T08:21:00,False,336304,4,32.76,1021,"[[106.791516, -6.352186], [106.791563, -6.3520...",0,0,0,KOTA DEPOK,8.48,820,5,4.47,Null,2019-07-30T08:20:00,Krukut Raya,0,Small,2019-07-30T01:22:44,1564449764942,1,7
1321588,0,[],Bekasi,ID,2019-10-05T02:35:24,222,Tue Jul 30 00:28:39 +0000 2019,1564446519770,20,Null,2019-07-30T08:33:00,False,359479,3,32.16,1180,"[[107.017011, -6.321264], [107.01703, -6.32126...",0,0,0,KAB. BEKASI,28.75,344,5,12.33,Null,2019-07-30T08:32:00,Letjen Suprapto,0,Small,2019-07-30T01:34:03,1564450443083,0,5
1321596,0,[],Bekasi,ID,2019-10-05T02:37:25,468,Tue Jul 30 01:26:28 +0000 2019,1564449988033,2,Null,2019-07-30T08:33:00,False,359722,4,32.75,696,"[[106.957727, -6.256052], [106.957754, -6.2558...",0,0,0,KOTA BEKASI,13.93,555,5,4.51,Null,2019-07-30T08:32:00,Arah Tol Jkt-Cikampek/Bekasi/Tol Purbaleunyi,-1,Small,2019-07-30T01:34:02,1564450442983,0,0
1321597,0,[],Bekasi,ID,2019-10-05T02:37:25,222,Tue Jul 30 00:28:39 +0000 2019,1564446519770,20,Null,2019-07-30T08:33:00,False,359729,3,32.75,1180,"[[107.017011, -6.321264], [107.01703, -6.32126...",0,0,0,KOTA BEKASI,28.75,344,5,12.33,Null,2019-07-30T08:32:00,Letjen Suprapto,0,Small,2019-07-30T01:34:03,1564450443083,0,5


In [0]:
## Export into CSV
df_alerts.to_csv('alerts.csv')
df_jams.to_csv('jams.csv')
df_irregularities.to_csv('irregularities.csv')

In [0]:
sc.stop()

In [0]:
## Pyspark General & Streaming
from operator import add, sub
from pyspark.streaming import StreamingContext
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, HiveContext
from pyspark.ml.feature import VectorAssembler, OneHotEncoderEstimator, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from operator import attrgetter
sc = SparkContext()
spark = SparkSession.builder \
                    .appName('DF_Finale') \
                    .master('spark://35.197.135.232:7077') \
                    .enableHiveSupport() \
                    .getOrCreate()
ssc = StreamingContext(sc, 1)
sqlContext = SQLContext(sc)
hiveContext = HiveContext(sc)

In [50]:
## Create Spark Dataframe and Data Exploration
### Jams
sc_jams = spark.read.csv('jams.csv', header = True, inferSchema = True)
sc_jams.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- created_time: timestamp (nullable = true)
 |-- delay: integer (nullable = true)
 |-- endNode: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- kode_kabupaten: double (nullable = true)
 |-- length: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- line: string (nullable = true)
 |-- nama_kabupaten: string (nullable = true)
 |-- pubMillis: long (nullable = true)
 |-- roadType: integer (nullable = true)
 |-- speed: double (nullable = true)
 |-- speedKMH: double (nullable = true)
 |-- startNode: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- street: string (nullable = true)
 |-- turnType: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uuid: integer (nullable = true)
 |-- peak: integer (nullable = true)



In [51]:
### Irregularities
sc_irregularities = spark.read.csv('irregularities.csv', header = True, inferSchema = True)
sc_irregularities.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- alertCount: integer (nullable = true)
 |-- alerts: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- created_time: timestamp (nullable = true)
 |-- delaySeconds: integer (nullable = true)
 |-- detectionDate: string (nullable = true)
 |-- detectionDateMillis: long (nullable = true)
 |-- driversCount: integer (nullable = true)
 |-- endNode: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- highway: boolean (nullable = true)
 |-- id: integer (nullable = true)
 |-- jamLevel: integer (nullable = true)
 |-- kode_kabupaten: double (nullable = true)
 |-- length: integer (nullable = true)
 |-- line: string (nullable = true)
 |-- nComments: integer (nullable = true)
 |-- nImages: integer (nullable = true)
 |-- nThumbsUp: integer (nullable = true)
 |-- nama_kabupaten: string (nullable = true)
 |-- regularSpeed: double (nullable = true)
 |-- seconds: integer (nullable = true)

In [52]:
### Alerts
sc_alerts = spark.read.csv('alerts.csv', header = True, inferSchema = True)
sc_alerts.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- confidence: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- created_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- kode_kabupaten: double (nullable = true)
 |-- location: string (nullable = true)
 |-- magvar: integer (nullable = true)
 |-- nThumbsUp: integer (nullable = true)
 |-- nama_kabupaten: string (nullable = true)
 |-- pubMillis: long (nullable = true)
 |-- reliability: integer (nullable = true)
 |-- reportRating: integer (nullable = true)
 |-- roadType: integer (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- street: string (nullable = true)
 |-- subtype: string (nullable = true)
 |-- type: string (nullable = true)
 |-- uuid: string (nullable = true)



In [53]:
sc_irregularities.show()

+---+----------+--------------------+------+-------+-------------------+------------+--------------------+-------------------+------------+-------+-------------------+-------+------+--------+--------------+------+--------------------+---------+-------+---------+----------------+------------+-------+--------+-----+---------+-------------------+--------------+-----+------+-------------------+----------------+------+--------------+
|_c0|alertCount|              alerts|  city|country|       created_time|delaySeconds|       detectionDate|detectionDateMillis|driversCount|endNode|           end_time|highway|    id|jamLevel|kode_kabupaten|length|                line|nComments|nImages|nThumbsUp|  nama_kabupaten|regularSpeed|seconds|severity|speed|startNode|         start_time|        street|trend|  type|         updateDate|updateDateMillis|isPeak|BK_driverCount|
+---+----------+--------------------+------+-------+-------------------+------------+--------------------+-------------------+--------

In [54]:
sc_jams.show()

+---+------+-------+-------------------+-----+--------------------+-------------------+--------------+------+-----+--------------------+--------------+-------------+--------+--------+--------+---------+-------------------+--------------------+--------+-----+----------+----+
|_c0|  city|country|       created_time|delay|             endNode|           end_time|kode_kabupaten|length|level|                line|nama_kabupaten|    pubMillis|roadType|   speed|speedKMH|startNode|         start_time|              street|turnType| type|      uuid|peak|
+---+------+-------+-------------------+-----+--------------------+-------------------+--------------+------+-----+--------------------+--------------+-------------+--------+--------+--------+---------+-------------------+--------------------+--------+-----+----------+----+
|  0| Bogor|     ID|2019-10-04 15:13:41|  267|         Cipaku Raya|2019-07-24 07:45:00|         32.01|   251|    4|[[106.80851, -6.6...|    KAB. BOGOR|1563927699475|       2|0

In [55]:
sc_alerts.show()

+---+------+----------+-------+-------------------+-------------------+--------------+--------------------+------+---------+--------------+-------------+-----------+------------+--------+-------------------+--------------------+--------------------+----+--------------------+
|_c0|  city|confidence|country|       created_time|           end_time|kode_kabupaten|            location|magvar|nThumbsUp|nama_kabupaten|    pubMillis|reliability|reportRating|roadType|         start_time|              street|             subtype|type|                uuid|
+---+------+----------+-------+-------------------+-------------------+--------------+--------------------+------+---------+--------------+-------------+-----------+------------+--------+-------------------+--------------------+--------------------+----+--------------------+
| 32| Depok|         0|     ID|2019-10-04 22:26:59|2019-07-29 08:14:00|         32.01|[106.90008, -6.41...|   243|        0|    KAB. BOGOR|1564361553000|          5|       

In [56]:
## Perform descriptive analytics
### df_jams
sc_jams.describe().toPandas().transpose() 

,0,1,2,3,4
summary,count,mean,stddev,min,max
_c0,105728,660821.484375,381514.85703135014,0,1321584
city,105728,None,None,Bekasi,Depok
country,105728,None,None,ID,ID
delay,105728,418.875,250.60305439056725,224,1578
endNode,79296,None,None,Akses Universitas Indonesia,Trans Yogi
kode_kabupaten,105728,32.529218750002244,0.33754929982892073,32.01,32.76
length,105728,1029.84375,702.5079979182083,154,2854
level,105728,3.390625,0.6025284583294301,2,4
line,105728,None,None,"[[106.739543, -6.570723], [106.739895, -6.5715...","[[107.016922, -6.268933], [107.01715, -6.26836..."


In [57]:
## Perform descriptive analytics
### df_irregularities
sc_irregularities.describe().toPandas().transpose() 

,0,1,2,3,4
summary,count,mean,stddev,min,max
_c0,289100,660788.1257142858,381513.7254044391,6,1321599
alertCount,289100,1.8514285714285714,6.358868551368053,0,30
alerts,289100,None,None,[],"[{'city': 'Depok', 'confidence': 1, 'country':..."
city,289100,None,None,Bekasi,Depok
country,289100,None,None,ID,ID
delaySeconds,289100,675.6171428571429,434.17120688564665,133,2575
detectionDate,289100,None,None,Fri Jul 26 01:21:51 +0000 2019,Wed Jul 24 23:48:53 +0000 2019
detectionDateMillis,289100,1.5642191882350999E12,1.981312468205362E8,1563926820964,1564449988033
driversCount,289100,16.62857142857143,12.326730332605345,2,55


In [58]:
## Perform descriptive analytics
### df_irregularities
sc_alerts.describe().toPandas().transpose() 

,0,1,2,3,4
summary,count,mean,stddev,min,max
_c0,118944,660828.625,381514.64994335483,32,1321523
city,118944,None,None,Bekasi,Depok
confidence,118944,0.0,0.0,0,0
country,118944,None,None,ID,ID
kode_kabupaten,118944,32.5059722222188,0.3287095422927691,32.01,32.76
location,118944,None,None,"[106.767263, -6.403455]","[107.030528, -6.20577]"
magvar,118944,174.19444444444446,104.71017638087814,3,352
nThumbsUp,118944,0.0,0.0,0,0
nama_kabupaten,118944,None,None,KAB. BEKASI,KOTA DEPOK


In [59]:
# Selecting Columns
## Jams
sc_jams_select = sc_jams.select('length', 'speed', 'speedKMH', 'delay', 'level', 'peak')
sc_jams_select.show()

+------+--------+--------+-----+-----+----+
|length|   speed|speedKMH|delay|level|peak|
+------+--------+--------+-----+-----+----+
|   251|0.802778|    2.89|  267|    4|   0|
|  1921|    4.15|   14.94|  263|    2|   0|
|  1238| 2.52778|     9.1|  360|    3|   1|
|  2061| 1.80833|    6.51|  935|    4|   1|
|   154|0.408333|    1.47|  346|    4|   1|
|  2312| 2.81389|   10.13|  564|    3|   1|
|  1166| 2.93889|   10.58|  264|    3|   0|
|   154|0.408333|    1.47|  346|    4|   1|
|  2061| 1.80833|    6.51|  935|    4|   1|
|  1546| 2.73611|    9.85|  383|    3|   1|
|   289|0.880556|    3.17|  270|    3|   0|
|   391| 1.33611|    4.81|  241|    4|   0|
|  2312| 2.81389|   10.13|  564|    3|   1|
|   463|0.858333|    3.09|  484|    4|   1|
|  1166| 2.93889|   10.58|  264|    3|   0|
|   600| 1.24167|    4.47|  382|    3|   1|
|   597| 1.59167|    5.73|  308|    3|   0|
|   525| 1.30278|    4.69|  343|    4|   1|
|  1001| 2.34722|    8.45|  325|    3|   0|
|   230|0.841667|    3.03|  250|

In [60]:
## Irregularities
sc_Irregularities_select = sc_irregularities.select('alertCount', 'delaySeconds', 'driversCount', 'jamLevel', 
                                                    'regularSpeed', 'length', 'seconds', 'severity', 'isPeak', 'BK_driverCount')
sc_Irregularities_select.show()

+----------+------------+------------+--------+------------+------+-------+--------+------+--------------+
|alertCount|delaySeconds|driversCount|jamLevel|regularSpeed|length|seconds|severity|isPeak|BK_driverCount|
+----------+------------+------------+--------+------------+------+-------+--------+------+--------------+
|        23|         237|          23|       3|       15.01|  1288|    389|       5|     0|             5|
|        23|         688|          15|       4|       25.59|  1341|    820|       5|     1|             3|
|        23|         357|          14|       3|       21.32|  1809|    552|       5|     0|             3|
|        23|         949|          10|       4|        16.0|   986|   1149|       5|     1|             2|
|        23|         141|          13|       3|       32.99|   722|    218|       5|     0|             3|
|        23|         165|          13|       3|       21.67|   792|    255|       5|     0|             3|
|        19|         828|          17

In [61]:
## Perform Correlations of df_jams
import six
for i in sc_jams_select.columns:
    if not(isinstance(sc_jams_select.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to Peak for ", i, sc_jams_select.stat.corr('peak',i)) ### data testnya bloman ada dibuka

Correlation to Peak for  length 0.26703558922439563
Correlation to Peak for  speed -0.23229098038247611
Correlation to Peak for  speedKMH -0.23229104367775502
Correlation to Peak for  delay 0.6243718391408499
Correlation to Peak for  level 0.23339256941752498
Correlation to Peak for  peak 1.0


In [62]:
## Perform Correlations of df_irregularities
for i in sc_Irregularities_select.columns:
    if not( isinstance(sc_Irregularities_select.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to isPeak for ", i, sc_Irregularities_select.stat.corr('isPeak',i)) ### data testnya bloman ada dibuka

Correlation to isPeak for  alertCount -0.03940702884404511
Correlation to isPeak for  delaySeconds 0.6741291636110427
Correlation to isPeak for  driversCount 0.34090129426748467
Correlation to isPeak for  jamLevel 0.4860839260185077
Correlation to isPeak for  regularSpeed -0.30002885873899654
Correlation to isPeak for  length 0.3597943168980714
Correlation to isPeak for  seconds 0.6829081829320648
Correlation to isPeak for  severity nan
Correlation to isPeak for  isPeak 1.0
Correlation to isPeak for  BK_driverCount 0.346116388348756


In [63]:
## Prepare Data for ML
### Jams
jams_vector = VectorAssembler(inputCols = ['length', 'speed', 
                                           'speedKMH', 'delay', 'level'], outputCol = 'features_jams')
jams_df = jams_vector.transform(sc_jams_select)
jams_df = jams_df.select(['features_jams', 'peak'])
jams_df.show(3)

### Irregularities
irregularities_vector = VectorAssembler(inputCols = ['alertCount', 'delaySeconds', 'driversCount', 'regularSpeed', 
                                                     'length', 'seconds', 'severity', 'jamLevel', 'BK_driverCount'], outputCol = 'features_irreg')
irregularities_df = irregularities_vector.transform(sc_Irregularities_select)
irregularities_df = irregularities_df.select(['features_irreg', 'isPeak'])
irregularities_df.show(3)

## Split into Train and Test
### Jams
splits_jams = jams_df.randomSplit([0.7, 0.3])
jams_train_df = splits_jams[0]
jams_test_df = splits_jams[1]

### Irregularities
splits_irregularities = irregularities_df.randomSplit([0.7, 0.3])
Irregularities_train_df = splits_irregularities[0]
Irregularities_test_df = splits_irregularities[1]

+--------------------+----+
|       features_jams|peak|
+--------------------+----+
|[251.0,0.802778,2...|   0|
|[1921.0,4.15,14.9...|   0|
|[1238.0,2.52778,9...|   1|
+--------------------+----+
only showing top 3 rows

+--------------------+------+
|      features_irreg|isPeak|
+--------------------+------+
|[23.0,237.0,23.0,...|     0|
|[23.0,688.0,15.0,...|     1|
|[23.0,357.0,14.0,...|     0|
+--------------------+------+
only showing top 3 rows



In [64]:
## Prepare train & test split data
### Jams
jams_train_df.show(5)
jams_test_df.show(5)

### Irregularities
Irregularities_train_df.show(5)
Irregularities_test_df.show(5)

+--------------------+----+
|       features_jams|peak|
+--------------------+----+
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
+--------------------+----+
only showing top 5 rows

+--------------------+----+
|       features_jams|peak|
+--------------------+----+
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
|[154.0,0.408333,1...|   1|
+--------------------+----+
only showing top 5 rows

+--------------------+------+
|      features_irreg|isPeak|
+--------------------+------+
|[0.0,133.0,10.0,2...|     0|
|[0.0,133.0,10.0,2...|     0|
|[0.0,133.0,10.0,2...|     0|
|[0.0,133.0,10.0,2...|     0|
|[0.0,133.0,10.0,2...|     0|
+--------------------+------+
only showing top 5 rows

+--------------------+------+
|      features_irreg|isPeak|
+--------------------+------+
|[0.0,133.0,10.0,2...|     0|
|[0.0,133.0,10.0,2...|     0|
|

In [65]:
## Logistic Regression
### Prepare (Irregularities)
logReg_Irreg = LogisticRegression(featuresCol = 'features_irreg', labelCol = 'isPeak', maxIter=10)
logReg_Model_Irreg = logReg_Irreg.fit(Irregularities_train_df)
print("Matrix Coefficient: " + str(logReg_Model_Irreg.coefficientMatrix))
print("Vector Intercept: " + str(logReg_Model_Irreg.interceptVector))

Matrix Coefficient: DenseMatrix([[ 1.45023694e-01,  1.60566262e-02,  2.55441504e-02,
              -7.99445407e-02, -1.91504112e-03,  1.24377613e-02,
               0.00000000e+00, -3.71140235e+00,  4.42106157e-02]])
Vector Intercept: [-2.137892661929801]


In [66]:
## Logistic Regression
### Prepare (Jams)
logReg_Jams = LogisticRegression(featuresCol = 'features_jams', labelCol = 'peak', maxIter=10)
logReg_Model_Jams = logReg_Jams.fit(jams_train_df)
print("Matrix Coefficient: " + str(logReg_Model_Jams.coefficientMatrix))
print("Vector Intercept: " + str(logReg_Model_Jams.interceptVector))

Matrix Coefficient: DenseMatrix([[ 0.01029594, -3.82055841, -1.06126721,  0.04911396, -3.00579445]])

Vector Intercept: [-1.0366095223924008]


In [67]:
### Logistic Regression
#### Prediction - Irregularities
predictions_irreg = logReg_Model_Irreg.transform(Irregularities_test_df)
predictions_irreg.select('isPeak', 'prediction', 'features_irreg', 'probability', 'rawPrediction').show(10)

#### Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_irreg = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='isPeak')
print('Test Accuracy', evaluator_irreg.evaluate(predictions_irreg))

+------+----------+--------------------+--------------------+--------------------+
|isPeak|prediction|      features_irreg|         probability|       rawPrediction|
+------+----------+--------------------+--------------------+--------------------+
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|     0|       0.0|[0.0,133.0,10.0,2...|[0.99998880902894...|[11.4003920693098...|
|   

In [68]:
### Logistic Regression
#### Prediction - Jams
predictions_jams = logReg_Model_Jams.transform(jams_test_df)
predictions_jams.select('peak', 'prediction', 'features_jams', 'probability', 'rawPrediction').show(10)

#### Evaluation
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_jams = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='peak')
print('Test Accuracy', evaluator_jams.evaluate(predictions_jams))

+----+----------+--------------------+--------------------+--------------------+
|peak|prediction|       features_jams|         probability|       rawPrediction|
+----+----------+--------------------+--------------------+--------------------+
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.408333,1...|[0.09105135171191...|[-2.3008649481658...|
|   1|       1.0|[154.0,0.40

In [0]:
# ## JOining between Prediction table and the dataset

# sc_Irregularities_select.join(predictions_irreg, "left")

# # df1($"Id") === df2($"Id"), you will get duplicate ID columns

In [0]:
# # Convert into pandas dataframe
# pandas_df = spark_df.select("*").toPandas()
# pandas_df